<a href="https://colab.research.google.com/github/Nasim-Mahmud/LSTM-projects/blob/main/lstm_word_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [201]:
paths = "/content/test_data.docs"
docs = ""
with open(paths, 'r') as doc:
  docs = doc.read()
docs

'About the Program\nWhat is the course fee for  Data Science Mentorship Program (DSMP 2023)\nThe course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.\nWhat is the total duration of the course?\nThe total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)\nWhat is the syllabus of the mentorship program?\nWe will be covering the following modules:\nPython Fundamentals\nPython libraries for Data Science\nData Analysis\nSQL for Data Science\nMaths for Machine Learning\nML Algorithms\nPractical ML\nMLOPs\nCase studies\nYou can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390\nWill Deep Learning and NLP be a part of this program?\nNo, NLP and Deep Learning both are not a part of this program’s curriculum.\nWhat if I miss a live session? Will I get a recording of the session?\nYes all our sessions are recorded, so eve

In [202]:
# Need to build a dataset based on the given text, so first we need to convert this strings into integers so that our model can work on it.
import tensorflow as tf
from keras.preprocessing.text import Tokenizer

In [203]:
tokenizer = Tokenizer()

In [204]:
# Assigning a integer to every number
tokenizer.fit_on_texts([docs])
# See the tokenized words
# tokenizer.word_index

In [205]:
input_seq = []
# Separating all the sentences based on line break.\
for sentence in docs.split("\n"):
  # Converting every sentence to a sequence using its designated integer values. Getting a numeric representation.
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  # Converting sequences to a dataset of given input and output
  for i in range(1, len(tokenized_sentence)):
    input_seq.append(tokenized_sentence[:i+1])

In [206]:
# input_seq

In [207]:
lengthOfLargestSeq = max([len(x) for x in input_seq])
lengthOfLargestSeq

57

In [208]:
# Using zero padding to make every sequence equal in length, Sparse vectorization.
from keras.preprocessing.sequence import pad_sequences
padded_input_seq = pad_sequences(input_seq, maxlen = lengthOfLargestSeq, padding= 'pre')
padded_input_seq

array([[  0,   0,   0, ...,   0,  93,   1],
       [  0,   0,   0, ...,  93,   1,  13],
       [  0,   0,   0, ...,   0,  11,   7],
       ...,
       [  0,   0,   0, ..., 279,  18, 280],
       [  0,   0,   0, ...,  18, 280, 281],
       [  0,   0,   0, ..., 280, 281, 282]], dtype=int32)

In [209]:
X = padded_input_seq[:, :-1]
X

array([[  0,   0,   0, ...,   0,   0,  93],
       [  0,   0,   0, ...,   0,  93,   1],
       [  0,   0,   0, ...,   0,   0,  11],
       ...,
       [  0,   0,   0, ...,   0, 279,  18],
       [  0,   0,   0, ..., 279,  18, 280],
       [  0,   0,   0, ...,  18, 280, 281]], dtype=int32)

In [210]:
y = padded_input_seq[:, -1]

In [211]:
X.shape

(863, 56)

In [212]:
y.shape

(863,)

In [213]:
numOfWords = len(tokenizer.word_index)

In [214]:
# Using one hot encoding
from keras.utils import to_categorical
y = to_categorical(y, num_classes= numOfWords + 1)
y.shape

(863, 283)

# **Building the LSTM architecture**

In [215]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [216]:
model = Sequential()

# Converting Sparse vectorization to Embedding, shape will be 100, input lenght will be the length of largest sequence -1.
model.add(Embedding(numOfWords + 1, 100, input_length= lengthOfLargestSeq - 1))

# Number of nodes in the gate of LSTM layer
model.add(LSTM(150))
model.add(Dense(numOfWords + 1, activation= "softmax"))

In [217]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [218]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 56, 100)           28300     
                                                                 
 lstm_4 (LSTM)               (None, 150)               150600    
                                                                 
 dense_4 (Dense)             (None, 283)               42733     
                                                                 
Total params: 221633 (865.75 KB)
Trainable params: 221633 (865.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [219]:
model.fit(X, y, epochs= 100)

Epoch 1/100
27/27 [==============================] - 5s 93ms/step - loss: 5.4573 - accuracy: 0.0637
Epoch 2/100
27/27 [==============================] - 3s 129ms/step - loss: 5.1055 - accuracy: 0.0776
Epoch 3/100
27/27 [==============================] - 3s 125ms/step - loss: 5.0220 - accuracy: 0.0776
Epoch 4/100
27/27 [==============================] - 3s 94ms/step - loss: 4.9900 - accuracy: 0.0776
Epoch 5/100
27/27 [==============================] - 2s 92ms/step - loss: 4.9319 - accuracy: 0.0776
Epoch 6/100
27/27 [==============================] - 2s 91ms/step - loss: 4.8511 - accuracy: 0.0834
Epoch 7/100
27/27 [==============================] - 5s 172ms/step - loss: 4.7019 - accuracy: 0.1159
Epoch 8/100
27/27 [==============================] - 3s 108ms/step - loss: 4.5015 - accuracy: 0.1344
Epoch 9/100
27/27 [==============================] - 2s 92ms/step - loss: 4.2795 - accuracy: 0.1703
Epoch 10/100
27/27 [==============================] - 2s 91ms/step - loss: 4.0401 - accuracy: 0.

In [220]:
import numpy as np

In [258]:
def word_predictor(text):
  for i in range(20):
    #tokenier
    tokenized_text = tokenizer.texts_to_sequences([text])[0]
    #padding
    padded_text = pad_sequences([tokenized_text], maxlen= lengthOfLargestSeq - 1, padding= "pre")
    #predict
    position = np.argmax(model.predict(padded_text, verbose=0))

    for word, index in tokenizer.word_index.items():
      if index == position:
        text = text + " " + word
  print(text)

In [259]:
word_predictor("A mail")

A mail follows a monthly subscription model where you have to make monthly payments of rs 799 month to end month to
